In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import matplotlib.pyplot as plt
import numpy as np
import csv
from PIL import Image
import time
import cv2 as cv

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())
print(torch.cuda.current_device())
torch.cuda.set_device(1)
#print(torch.cuda.current_device())

Using cuda device
Device: cuda
Current cuda device: 0
Count of using GPUs: 2
0


In [2]:
xs_0=list()
xs_0_=list()
xs_1=list()
ys_0=list()
data_number= 200
batch_size = 20
epochs = 3000
#100장정도.. 램 가능
with open ('data/Data_narrow.txt', 'rt' ) as r_n:
    target_r = r_n.readline()
    for j in range(data_number): 
        print(j)
        xs_0=list()
        for i in range(20): 
            temp_red=list()
            temp_blue=list()
            temp_green=list()
            temp_red_=list()
            temp_blue_=list()
            temp_green_=list()
            
            r=open ('data/'+str(i+20*j)+'_building_r.csv', 'r' )
            rdr=csv.reader(r)
            for target in rdr:
                temp_red.append(target)
                
            r=open ('data/'+str(i+20*j)+'_building_g.csv', 'r' )
            rdr=csv.reader(r)
            for target in rdr:
                temp_green.append(target)
                
            r=open ('data/'+str(i+20*j)+'_building_b.csv', 'r' )
            rdr=csv.reader(r)
            for target in rdr:
                temp_blue.append(target)
                
            temp_red=np.array(temp_red, dtype=np.float64)
            temp_greed=np.array(temp_green, dtype=np.float64)
            temp_blue=np.array(temp_blue, dtype=np.float64)
            #####
            r=open ('data/'+str(i+20*j)+'_r.csv', 'r' )
            rdr=csv.reader(r)
            for target in rdr:
                temp_red_.append(target)

            r=open ('data/'+str(i+20*j)+'_g.csv', 'r' )
            rdr=csv.reader(r)
            for target in rdr:
                temp_green_.append(target)

            r=open ('data/'+str(i+20*j)+'_b.csv', 'r' )
            rdr=csv.reader(r)
            for target in rdr:
                temp_blue_.append(target)

            temp_red_=np.array(temp_red_, dtype=np.float64)
            temp_greed_=np.array(temp_green_, dtype=np.float64)
            temp_blue_=np.array(temp_blue_, dtype=np.float64)
            ####
            xs_1=np.stack((temp_red, temp_green, temp_blue, temp_red_, temp_green_, temp_blue_), axis=0)
            xs_1=list(xs_1) 
            xs_0.append(xs_1)
            
            target_r = r_n.readline()
            if i<20:
                A_r,B_r,C_r,D_r,E_r,F_r,G_r,H_r,I_r,J_r,K_r=target_r.split(',')  

                if i==0:
                    pass

                else:
                    if i==1:
                        ys_0.append([float(E_r)])

                    else:

                        ys_0=ys_0[:-1]+[ys_0[-1]+[float(E_r)]]

        xs_0_.append(xs_0)

x_train_ = np.array(xs_0_, dtype=np.float64)
y_train_ = np.array(ys_0, dtype=np.float64)
#y_train = np.zeros_like(y_train)
x_train = torch.from_numpy(x_train_[:int(data_number/2)]).cuda()
y_train = torch.from_numpy(y_train_[:int(data_number/2)]).cuda()
y_test = torch.from_numpy(y_train_[int(data_number/2):]).cuda()
x_test = torch.from_numpy(x_train_[int(data_number/2):]).cuda()
#print(y_train)
xs_0=list()
xs_0_=list()
xs_1=list()

print(x_train.shape,y_train.shape)
dataset = TensorDataset(x_train, y_train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
#print(x_test.shape,y_train[100:].shape)
print(x_test.shape,y_test.shape)
#print(x_test.shape,y_train.shape)
dataset_test = TensorDataset(x_test, y_test)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
xs_0=list()
xs_0_=list()
xs_1=list()
ys_0=list()
temp_red=list()
temp_blue=list()
temp_green=list()

print("data_ready")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
torch.Size([100, 20, 6, 100, 200]) torch.Size([100, 19])
torch.Size([100, 20, 6, 100, 200]) torch.Size([100, 19])
data_ready


In [3]:
class LRCN(nn.Module):
    def __init__(self):
        super(LRCN, self).__init__()
        self.layer_1 = nn.Conv2d(3,15,5,stride=2,dtype=torch.float64)
        self.layer_2 = nn.Conv2d(15,13,5,stride=2,dtype=torch.float64)
        self.layer_3 = nn.Conv2d(13,7,5,stride=2,dtype=torch.float64)
        self.layer_4 = nn.Conv2d(7,3,5,stride=2,dtype=torch.float64)
        self.layer_5 = nn.LSTM(81,40,2, batch_first = True,dtype=torch.float64)
        
        self.layer_6 = nn.Conv2d(3,15,5,stride=2,dtype=torch.float64)
        self.layer_7 = nn.Conv2d(15,13,5,stride=2,dtype=torch.float64)
        self.layer_8 = nn.Conv2d(13,7,5,stride=2,dtype=torch.float64)
        self.layer_9 = nn.Conv2d(7,3,5,stride=2,dtype=torch.float64)
        self.layer_10 = nn.LSTM(81,40,2, batch_first = True,dtype=torch.float64)
        
        self.layer_11 = nn.Linear(80, 1,dtype=torch.float64)
        
        self.batch_size = batch_size
        self.length = x_train.shape[1]
        self.relu = nn.ReLU()
        self.glob_flag=0
        self.batch_number=0
        self.flag_1=0
        self.flag_2=0
        self.switch = torch.nn.Parameter(torch.tensor(0.5,dtype=torch.float64), requires_grad=True)
        #self.switch.requires_grad=True
        
        
    def forward(self, x):
        #print(x.shape)

        x_1=x[:,:,0:3]
        x_2=x[:,:,3:]
            
        
        x_1 = x_1.reshape((-1, 3, 100, 200))
        #print(x_1.shape)
        x_1 = self.layer_1(x_1)
        x_1 = self.relu(x_1)
        x_1 = self.layer_2(x_1)
        x_1 = self.relu(x_1)
        x_1 = self.layer_3(x_1)
        x_1 = self.relu(x_1)
        x_1 = self.layer_4(x_1)
        x_1 = self.relu(x_1)
        #print(x_1.shape)
        x_1 = x_1.reshape((self.batch_size, self.length, x_1.shape[-3],x_1.shape[-2],x_1.shape[-1]))
        x_1 = x_1.reshape((self.batch_size, self.length, x_1.shape[-3]*x_1.shape[-2]*x_1.shape[-1]))
        x_1, (h_n, c_n) = self.layer_5(x_1)
        
        x_2 = x_2.reshape((-1, 3, 100, 200))
        x_2 = self.layer_6(x_2)
        x_2 = self.relu(x_2)
        x_2 = self.layer_7(x_2)
        x_2 = self.relu(x_2)
        x_2 = self.layer_8(x_2)
        x_2 = self.relu(x_2)
        x_2 = self.layer_9(x_2)
        x_2 = self.relu(x_2)
        x_2 = x_2.reshape((self.batch_size, self.length, x_2.shape[-3],x_2.shape[-2],x_2.shape[-1]))
        x_2 = x_2.reshape((self.batch_size, self.length, x_2.shape[-3]*x_2.shape[-2]*x_2.shape[-1]))
        x_2, (h_n, c_n) = self.layer_10(x_2)
        
        if self.flag_1 == 1:
            x_1 = torch.zeros_like(x_1)
        elif self.flag_2 == 1:
            x_2 = torch.zeros_like(x_2)
            
        #print(x_1.shape,x_2.shape)
        x_1 = self.switch*x_1
        x_2 = (1-self.switch)*x_2
        self.x_3 = torch.cat((x_1,x_2),dim=2)
        
        
        x_3 = self.layer_11(self.x_3)
        #print(x_3.shape)
        x_3 = torch.squeeze(x_3,dim=2)
        #print(x_3.shape)
        #print(x.shape,h_n.shape,c_n.shape)
        #print(x.shape)
        return x_3[:,1:]

model = LRCN().cuda()
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer=optimizer,
                                        lr_lambda=lambda epoch: 1 ,
                                        last_epoch=-1,
                                        verbose=False)
def image_out(feature,batch,name):
    width = int(feature.grad.shape[-1])
    #print(width_)
    #print(feature.grad)
    abs_grad = np.abs(feature.grad.detach().cpu().numpy())
    abs_grad = abs_grad[:,1:]
    min_= np.min(abs_grad)
    abs_grad=abs_grad-min_
    max_ = np.max(abs_grad)
    result_ = np.around(((255*abs_grad)/max_)).astype(np.uint8)
    print(np.sum(result_[:,:,0:20]),np.sum(result_[:,:,20:]))
    
    width = int(feature.shape[-1])
    #print(width_)
    #print(feature.grad)
    abs_grad = np.abs(feature.detach().cpu().numpy())
    abs_grad = abs_grad[:,1:]
    min_= np.min(abs_grad)
    abs_grad=abs_grad-min_
    max_ = np.max(abs_grad)
    result_ = np.around(((255*abs_grad)/max_)).astype(np.uint8)
    print(np.sum(result_[:,:,0:20]),np.sum(result_[:,:,20:]))
    


def train(dataloader, model, loss, optimizer):
    loss_sum=0
    total_number=len(dataloader)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss_result = loss(pred, y)
        #print(pred.shape,y.shape)
        optimizer.zero_grad()
        if total_number == batch + 1:
            model.x_3.retain_grad()
        loss_result.backward()
        optimizer.step()
        loss_sum+=loss_result
    loss_sum/(batch+1)
    
    print("train_loss",loss_sum)
    
    if model.glob_flag==1:
        model.glob_flag=0
        #image_out(model.x_3, 0,"test_")
        """
        model.flag_1=1
        
        loss_sum=0
        total_number=len(dataloader)
        for batch, (X, y) in enumerate(dataloader):
            pred = model(X)
            loss_result = loss(pred, y)
            #print(pred.shape,y.shape)
            loss_sum+=loss_result
        loss_sum/(batch+1)
        print("train_loss_flag_1",loss_sum)
        model.flag_1=0
        
        model.flag_2=1
        loss_sum=0
        total_number=len(dataloader)
        for batch, (X, y) in enumerate(dataloader):
            pred = model(X)
            loss_result = loss(pred, y)
            #print(pred.shape,y.shape)
            loss_sum+=loss_result
        loss_sum/(batch+1)
        print("train_loss_flag_2",loss_sum)
        model.flag_2=0
        """
        for i, para in enumerate(model.parameters()):
            if 33 == i:
                #print(para.shape)
                #print(torch.sum(para[0,:40]),torch.sum(para[0,40:]),torch.sum(torch.abs(para[0,:40])),torch.sum(torch.abs(para[0,40:])))
                #print(model.switch)
                with open("switch.csv",'a',newline='') as aa:
        
                    #print (im.size) #Get the width and hight of the image for iterating over
                    writer_a = csv.writer(aa, delimiter=',')
                    #print(model.switch.item())
                    writer_a.writerow([model.switch.item()])
    #if model.glob_flag==100:
        #image_out(model.a,0,"training_LRCN_1_")
        #model.glob_flag=0
        #torch.save(model,"model_1.pt")
        #time.sleep(1)

def test(dataloader, model, loss):
    loss_sum=0
    total_number=len(dataloader)
    for batch, (X,y) in enumerate(dataloader):
        pred = model(X)
        loss_result = loss(pred, y)
        #print(pred.shape,y.shape)
        #if total_number == batch + 1:
        #    optimizer.zero_grad()
        #    model.a.retain_grad()
        #loss_result.backward()
        loss_sum+=loss_result
    loss_sum/(batch+1)
    print("test_loss",loss_sum)
    

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(dataloader, model, loss, optimizer)
    test(dataloader_test, model, loss)
    model.glob_flag+=1
    scheduler.step()
    
print("Done!")

Epoch 1
-------------------------------
train_loss tensor(70.4878, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(72.4240, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 2
-------------------------------
train_loss tensor(69.2646, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(71.4187, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 3
-------------------------------
train_loss tensor(68.3604, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(70.5504, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 4
-------------------------------
train_loss tensor(67.5076, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(69.6820, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 5
-------------------------------
train_loss tensor(66.6575, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
tes

test_loss tensor(27.1539, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 39
-------------------------------
train_loss tensor(25.3011, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(26.3592, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 40
-------------------------------
train_loss tensor(24.5622, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(25.6104, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 41
-------------------------------
train_loss tensor(23.8640, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(24.8941, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 42
-------------------------------
train_loss tensor(23.1988, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(24.2342, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 43
-------------------------------

train_loss tensor(11.8576, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(12.7359, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 77
-------------------------------
train_loss tensor(11.6589, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(12.5317, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 78
-------------------------------
train_loss tensor(11.4718, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(12.3351, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 79
-------------------------------
train_loss tensor(11.2894, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(12.1523, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 80
-------------------------------
train_loss tensor(11.1167, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(11.9764, device='cuda:

train_loss tensor(7.3719, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(8.2219, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 115
-------------------------------
train_loss tensor(7.3100, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(8.1193, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 116
-------------------------------
train_loss tensor(7.2446, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(8.0576, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 117
-------------------------------
train_loss tensor(7.1853, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(8.0136, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 118
-------------------------------
train_loss tensor(7.1078, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(7.9324, device='cuda:1', dt

train_loss tensor(5.6313, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(6.4545, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 153
-------------------------------
train_loss tensor(5.6097, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(6.3720, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 154
-------------------------------
train_loss tensor(5.5871, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(6.3260, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 155
-------------------------------
train_loss tensor(5.5465, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(6.3467, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 156
-------------------------------
train_loss tensor(5.5260, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(6.2873, device='cuda:1', dt

...

train_loss tensor(1.0421, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(4.8209, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 1673
-------------------------------
train_loss tensor(1.0413, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(4.7950, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 1674
-------------------------------
train_loss tensor(1.0358, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(4.7632, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 1675
-------------------------------
train_loss tensor(1.0360, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(4.8351, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 1676
-------------------------------
train_loss tensor(1.0320, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(4.7848, device='cuda:1'

train_loss tensor(1.0267, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(4.8950, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 1711
-------------------------------
train_loss tensor(1.0064, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(4.8396, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 1712
-------------------------------
train_loss tensor(1.0156, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(4.8102, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 1713
-------------------------------
train_loss tensor(1.0501, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(4.9042, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 1714
-------------------------------
train_loss tensor(1.0392, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(4.8565, device='cuda:1'

train_loss tensor(0.9797, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(4.8706, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 1749
-------------------------------
train_loss tensor(0.9682, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(4.9274, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 1750
-------------------------------
train_loss tensor(0.9915, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(4.8883, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 1751
-------------------------------
train_loss tensor(0.9845, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(4.9447, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 1752
-------------------------------
train_loss tensor(0.9820, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(4.9250, device='cuda:1'

train_loss tensor(0.9535, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(4.9346, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 1787
-------------------------------
train_loss tensor(0.9540, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(4.9572, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 1788
-------------------------------
train_loss tensor(0.9502, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(4.9125, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 1789
-------------------------------
train_loss tensor(0.9582, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(4.9277, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
Epoch 1790
-------------------------------
train_loss tensor(0.9654, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
test_loss tensor(4.9136, device='cuda:1'

KeyboardInterrupt: 